### Create CNN model and optimize it using keras tuner

In [16]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.18.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
(train_img,train_labels),(test_img,test_labels) = fashion_mnist

In [6]:
train_img.shape

(60000, 28, 28)

In [7]:
train_img = train_img/255.0
test_img = test_img/255.0

In [10]:
train_img[0].shape

(28, 28)

In [11]:
train_img = train_img.reshape(len(train_img),28,28,1)
test_img = test_img.reshape(len(test_img),28,28,1)

In [13]:
train_img.shape

(60000, 28, 28, 1)

In [25]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter',min_value=32,max_value = 128,step = 16),
          kernel_size=hp.Choice('conv_1_kernel',values = [3,5]),
          activation = 'relu',
          input_shape = (28,28,1)
      ),
      keras.layers.Conv2D(
          filters = hp.Int('conv_2_filter',min_value = 32,max_value = 128,step=16),
          kernel_size = hp.Choice('conv_2_kernel',values = [3,5]),
          activation = 'relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units = hp.Int('dense_1_units',min_value=32,max_value=128,step = 16),
          activation= 'relu'
      ),
      keras.layers.Dense(10,activation='softmax')
  ])


  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  return model

In [26]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [27]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory = 'output',project_name = 'Mnist Fashion')

In [28]:
tuner_search.search(train_img,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.9054999947547913

Best val_accuracy So Far: 0.9054999947547913
Total elapsed time: 00h 03m 14s


In [29]:
model = tuner_search.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [30]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 32)          │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 22, 22, 80)          │          23,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 38720)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 96)                  │       3,717,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             970 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,742,138 (14.28 MB)

 Trainable params: 3,742,138 (14.28 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(train_img,train_labels,epochs = 10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - accuracy: 0.9247 - loss: 0.1999 - val_accuracy: 0.9120 - val_loss: 0.2474
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9477 - loss: 0.1369 - val_accuracy: 0.9098 - val_loss: 0.2815
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9637 - loss: 0.0971 - val_accuracy: 0.9108 - val_loss: 0.3195
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9748 - loss: 0.0681 - val_accuracy: 0.9012 - val_loss: 0.3290
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9811 - loss: 0.0506 - val_accuracy: 0.9107 - val_loss: 0.3903
Epoch 9/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9860 - loss: 0.0392 - val_accuracy: 0.9115 - val_loss: 0.4229
Epoch 10/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9888 - loss: 0.0300 - val_accuracy: 0.9210 - val_loss: 0.4801


In [ ]:
;